In [13]:
from rajvi_mapbox_api import MapboxAPI
import requests
from urllib.request import urlopen
import json
from shapely.geometry import shape, Polygon

import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
plt.style.use('default')
import geopandas as gpd
import geodatasets
import ast
from shapely import wkt
import folium
from pyproj import Geod

In [14]:
branches = pd.read_csv('../data/clean/branches.csv')
populationDemographics = pd.read_csv('../data/clean/population_demographics.csv')
branchRadii = gpd.read_file('../data/clean/branches.csv')
api = MapboxAPI()
geod=Geod(ellps='WGS84')

In [15]:
# prep branchRadii dataframe for manipulation
branchRadii = branchRadii.set_crs('EPSG:4326')
branchRadii[['LATITUDE', 'LONGITUDE']] = [ast.literal_eval(x)[:2] for x in branchRadii['LOCATION']]
branchRadii.loc[:, 'LATITUDE'] = pd.to_numeric(branchRadii.loc[:, 'LATITUDE'])
branchRadii.loc[:, 'LONGITUDE'] = pd.to_numeric(branchRadii.loc[:, 'LONGITUDE'])
branchRadii.head()
branchRadii = branchRadii.set_index('BRANCH')
branchRadii = branchRadii.drop(['HOURS OF OPERATION', 'ADDRESS', 'CITY', 'STATE','ZIP','PHONE','WEBSITE',
                                          'PERMANENTLY_CLOSED'], axis=1)
branchRadii

,LOCATION,geometry,LATITUDE,LONGITUDE
BRANCH,,,,
Albany Park,"(41.97557881655979, -87.71361314512697)",None,41.975579,-87.713613
Altgeld,"(41.65719847558056, -87.59883829075173)",None,41.657198,-87.598838
Archer Heights,"(41.80110836194246, -87.72648385568911)",None,41.801108,-87.726484
Austin,"(41.88917312206642, -87.7658203582574)",None,41.889173,-87.765820
Austin-Irving,"(41.95312267684315, -87.77928489355646)",None,41.953123,-87.779285
...,...,...,...,...
West Pullman,"(41.67790647382097, -87.6431683153105)",None,41.677906,-87.643168
West Town,"(41.89594730478282, -87.66829305460979)",None,41.895947,-87.668293
"Whitney M. Young, Jr.","(41.7510184591363, -87.61495065361933)",None,41.751018,-87.614951


In [16]:
# make and prep censusTracts data frame for manipulation

censusTracts = populationDemographics[['geoid','geometry']]
censusTracts = gpd.GeoDataFrame(
    censusTracts.loc[:, [c for c in censusTracts.columns if c != "geometry"]],
    geometry=gpd.GeoSeries.from_wkt(censusTracts["geometry"]),
    crs="epsg:4326",
    )

censusAreas = []
for index, row in censusTracts.iterrows():
    poly_area, poly_perimeter = geod.geometry_area_perimeter(row['geometry'])
    poly_area = poly_area / -10**6
    censusAreas.append(poly_area)

censusTracts['area in degrees'] = censusAreas
censusTracts

,geoid,geometry,area in degrees
0,17031823303,"POLYGON ((-87.72096 41.68395, -87.72030 41.683...",0.000001
1,17031823304,"MULTIPOLYGON (((-87.72004 41.68397, -87.72006 ...",0.352650
2,17031740400,"POLYGON ((-87.72008 41.69127, -87.71937 41.691...",2.782659
3,17031740300,"POLYGON ((-87.72043 41.69857, -87.72007 41.698...",1.319688
4,17031822802,"MULTIPOLYGON (((-87.72075 41.70204, -87.72075 ...",0.000016
...,...,...,...
864,17031320400,"POLYGON ((-87.62764 41.88206, -87.62759 41.882...",0.803338
865,17031320101,"POLYGON ((-87.62086 41.88188, -87.62082 41.882...",0.490280
866,17031081403,"MULTIPOLYGON (((-87.62431 41.88911, -87.62431 ...",0.595628
867,17031081402,"POLYGON ((-87.61754 41.89345, -87.61716 41.893...",0.577679


In [17]:
# prep empty data frame for individual lib data

chosenLib =pd.DataFrame
chosenLib = pd.DataFrame(columns=['Census Tract','geometry'])
chosenLib.reset_index(inplace = True)

In [27]:
empty = []
geod=Geod(ellps='WGS84')
def single_lib_overlap(branchName, travelTime):
    '''call this function with the library name as a string and travel time as an int to get a data frame
    for the chosen library with the percent of each census tract covered in the entered minute radius'''
    
    api_response = api.get_data((branchRadii.loc[branchName,'LATITUDE'],branchRadii.loc[branchName,'LONGITUDE']), travelTime)
    branchRadii.loc[branchName,'geometry'] = shape(api_response)
    chosenLib =pd.DataFrame
    chosenLib = pd.DataFrame(columns=['Census Tract','geometry'])
    chosenLib.reset_index(inplace = True)
    
    for index, row in censusTracts.iterrows():
        if ((branchRadii.intersection(row['geometry']))[0]).is_empty:
            empty=[]
            empty.append("none")
        else:
            temp = pd.DataFrame({"Census Tract": row['geoid'],"geometry": [(branchRadii.intersection(row['geometry'])[0])]}, index=[0])
            chosenLib=pd.concat([chosenLib,temp])
   
    return chosenLib

for index, row in areaCalculations.iterrows():
    poly_area, poly_perimeter = geod.geometry_area_perimeter(row['geometry'])
    poly_area = poly_area / -10**6
    censusAreas.append(poly_area)

IndentationError: unexpected indent (1623531582.py, line 24)

In [29]:
single_lib_overlap('Albany Park', 5)

/opt/tljh/user/lib/python3.9/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)


,index,Census Tract,geometry,area overlap
0,NaN,17031140200,"MULTIPOLYGON (((-87.713613 41.97142, -87.71415...",0.009570
0,NaN,17031831800,"POLYGON ((-87.718838 41.975579, -87.716431 41....",0.407431


In [ ]:
for index, row2 in censusTracts.iterrows():
            if row2['geoid'] == row['Census Tract']:
                overlappingPercents.append(poly_area/row2['area in degrees'])
                
for index, row in chosenLib.iterrows():
        poly_area, poly_perimeter = geod.geometry_area_perimeter(row['geometry'])
        poly_area = poly_area / -10**6
        chosenLib['percent overlap'] = overlappingPercents
        
        for index, row2 in censusTracts.iterrows():
                if row2['geoid'] == row['CensusTract']:
                    percentOverlap = poly_area/row2['area in degrees']
                    overlappingPercents.append(percentOverlap)
        chosenLib['area overlap'] = overlappingAreas
        chosenLib['percent overlap'] = overlappingPercents